In [12]:
import pandas as pd
import glob
import os

work_dir = './'
csv_files = glob.glob(os.path.join(work_dir, '20*.csv'))

df = pd.concat((pd.read_csv(f) for f in csv_files), ignore_index=True)
df.to_csv('aggregated-step1.csv', index=False, float_format='%.8f')

pd.options.display.float_format = '{:.8f}'.format
df.head(3)

,Date(UTC),Pair,Side,Price,Executed,Amount,Fee
0,2020-12-19 01:12:17,BNBUSDT,SELL,31.81280000,3.1500000000BNB,100.21032000USDT,0.0023624900BNB
1,2020-12-08 07:25:16,UNFIBNB,SELL,0.49000000,6.4000000000UNFI,3.13600000BNB,0.0023520000BNB
2,2020-12-08 02:39:20,UNFIBNB,BUY,0.40882000,6.4000000000UNFI,2.61644800BNB,0.0019536800BNB


In [14]:
import pandas as pd
import re

pd.options.display.float_format = '{:.8f}'.format

def split_column(value):  
    match = re.match(r'^([\d\.]*)([a-zA-Z]*)', value)
    value_number = match.group(1)
    value_coin = match.group(2)
    return value_number, value_coin

df = pd.read_csv('aggregated-step1.csv')

df[['AmVal', 'AmCoin']] = df['Amount'].apply(split_column).apply(pd.Series)
df[['FeeVal', 'FeeCoin']] = df['Fee'].apply(split_column).apply(pd.Series)
df[['ExecVal', 'ExecCoin']] = df['Executed'].apply(split_column).apply(pd.Series)

df = df.drop(columns=['Amount', 'Fee', 'Executed'])

df.to_csv('aggregated-step2.csv', index=False, float_format='%.8f')

df.head(3)

,Date(UTC),Pair,Side,Price,AmVal,AmCoin,FeeVal,FeeCoin,ExecVal,ExecCoin
0,2020-12-19 01:12:17,BNBUSDT,SELL,31.81280000,100.21032000,USDT,0.0023624900,BNB,3.1500000000,BNB
1,2020-12-08 07:25:16,UNFIBNB,SELL,0.49000000,3.13600000,BNB,0.0023520000,BNB,6.4000000000,UNFI
2,2020-12-08 02:39:20,UNFIBNB,BUY,0.40882000,2.61644800,BNB,0.0019536800,BNB,6.4000000000,UNFI


In [18]:
import pandas as pd

pd.options.display.float_format = '{:.8f}'.format

df = pd.read_csv('aggregated-step2.csv')

df['AmValOrig'] = df['AmVal']
df['AmValExpect'] = df['Price'] * df['ExecVal']
df['AmValDiff'] = df['AmValExpect'] - df['AmValOrig']
df['AmValDiffPrc'] = (df['AmValDiff'] / df['AmValExpect']) * 100

df = df.drop(columns=['AmVal'])

df.to_csv('aggregated-step3.csv', index=False, float_format='%.8f')

df.head(3)

,Date(UTC),Pair,Side,Price,AmCoin,FeeVal,FeeCoin,ExecVal,ExecCoin,AmValOrig,AmValExpect,AmValDiff,AmValDiffPrc
0,2020-12-19 01:12:17,BNBUSDT,SELL,31.81280000,USDT,0.00236249,BNB,3.15000000,BNB,100.21032000,100.21032000,0.00000000,0.00000000
1,2020-12-08 07:25:16,UNFIBNB,SELL,0.49000000,BNB,0.00235200,BNB,6.40000000,UNFI,3.13600000,3.13600000,0.00000000,0.00000000
2,2020-12-08 02:39:20,UNFIBNB,BUY,0.40882000,BNB,0.00195368,BNB,6.40000000,UNFI,2.61644800,2.61644800,0.00000000,0.00000000


In [23]:
import pandas as pd

pd.options.display.float_format = '{:.8f}'.format

filter = 'MANA'
f = pd.read_csv('aggregated-step3.csv')
fdf = df[df['Pair'].str.startswith(filter)]

fdf.head(3)

,Date(UTC),Pair,Side,Price,AmCoin,FeeVal,FeeCoin,ExecVal,ExecCoin,AmValOrig,AmValExpect,AmValDiff,AmValDiffPrc
40,2021-10-30 15:05:14,MANABTC,SELL,0.00004177,BTC,0.00014836,BNB,40.00000000,MANA,0.00167080,0.00167080,0.00000000,0.00000000
41,2021-10-30 12:05:17,MANABTC,SELL,0.00002800,BTC,0.00002463,BNB,10.00000000,MANA,0.00028000,0.00028000,0.00000000,0.00000000
42,2021-10-30 12:04:58,MANABTC,SELL,0.00002792,BTC,0.00001497,BNB,6.00000000,MANA,0.00016752,0.00016752,0.00000000,0.00000000
